In [2]:
import os
import string
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
### Lower characters
def lowerChar(ip):
    s=""
    for ch in ip:
        ch=ch.lower()
        s+=ch
    return s

In [4]:
#### Remove Stop words

def removeStopwords(temp):
    stoWords=set(stopwords.words('english'))
    s=""
    ans=[]
    for h in temp:
        if not h in stoWords:
            ans.append(h)
    return ans 

In [5]:
#--------------------------------------TOKENIZATION-----------------------------#

def tokenize_ipStr(ip):   
    temp=word_tokenize(ip)
    return temp  

In [6]:
### Remove Punctuations
def removePuntuations(ip):
    test_str = ip.translate(str.maketrans('', '', string.punctuation))
    return test_str
    

In [7]:
#### Remove White Spaces
def removeWS(ip):
    s1=" ".join(ip.split())
    return s1

In [8]:
## Preprocessing
def preprocess(ip):
    # call lower function
    s=lowerChar(ip)
    s=removePuntuations(s)
    s=removeWS(s)
    # call tokenize file function
    temp = tokenize_ipStr(s)
    #Call remove Stop words funtions
    #print("\n\n\n")
    s=removeStopwords(temp)
    
    return s

In [9]:
#### AND Function

def andFunc(l1,l2):
    lenl1 = len(l1)
    lenl2 = len(l2)
    i=0
    j=0
    ansList = []
    cmprs = 0
    while(i<lenl1 and j<lenl2):
        if(l1[i] == l2[j]):
            ansList.append(l1[i])
            i=i+1
            j=j+1
        elif(l1[i] < l2[j]):
            i=i+1
        else:
            j=j+1

        cmprs=cmprs + 1

    return ansList,cmprs ##### MERGING

In [10]:
##### Bigram Inverted Index from pickle #####

def bigramInvIndex():
    with open('D:/SEMESTER-2/IR-Ass1/file_bigrams.pkl', 'rb') as file:
        
        # Call load method to deserialze
        dict = pickle.load(file)
    return dict

In [11]:
###### Positional Inverted Index from pickle #######

def positionalInvIndex():
    with open('D:/SEMESTER-2/IR-Ass1/file_positional.pkl', 'rb') as file:
        
        # Call load method to deserialze
        myvar = pickle.load(file)
    return myvar

In [12]:
doctIdDict={}
for i in range(1,1401):
    s=""
    if(i>=1 and i<=9):
        s="000"
    elif(i>=10 and i<=99):
        s="00"
    if(i>=100 and i<=999):
        s="0"
    text="cranfield"+s+"{x}".format(x=i)
    doctIdDict[i]=text


In [13]:
####### User Input ###########

sss=["null"]
def userInput(queryCount):

    print("Enter Query: ")
    ip = input()
    sss[0]=ip
    ans_temp = preprocess(ip) ##### Preprocess
    
    ans=[]

    for i in range(0,len(ans_temp)-1):
        ans.append(ans_temp[i]+" "+ans_temp[i+1])
    
    print("Tokens :", ans)
    ans_list=[]
    dict = bigramInvIndex()
    if(len(ans)==0):
        ans_list=[]
    elif(len(ans)==1):
        if dict.get(ans[0]) is not None:
            ans_list=dict[ans[0]]
        else:
            ans_list=[]

    else:
        for i in range(0,len(ans)-1):
            word1=ans[i]
            word2=ans[i+1]
            comp=0
            opList=[]
            if dict.get(word1) is not None:
                list1=dict[word1]
            else:
                list1 = []
            if dict.get(word2) is not None:
                list2=dict[word2]
            else:
                list2 = []

            if(len(ans_list)!=0):
                list1=ans_list
            opList,cmpr=andFunc(list1, list2)
            ans_list=opList

    ## Doc Names ##
    ans_doc_name=[]
    for i in ans_list:
        ans_doc_name.append(doctIdDict[i])
    ans_doc_name2=[]
    # for i in ans_list2:
    #     ans_doc_name2.append(doctIdDict[i])
        
    ######## Output #########
    queryCount=queryCount+1
    print("No of documents retrieved for query "+str(queryCount)+" using bigram inverted index: ", len(ans_list))
    print("Names of document retrieved for query "+str(queryCount)+" using bigram inverted index: ",ans_doc_name)

In [25]:
def userIP(queryCount):
    posDict = positionalInvIndex()
    s=sss[0]
    token=preprocess(s)
    common_docs=[]
    for i in range(0,len(token)-1):
        if posDict.get(token[i]) is not None:
            temp_list1=posDict[token[i]]
        else:
            temp_list1=[{}]
        if posDict.get(token[i+1]) is not None:
            temp_list2=posDict[token[i+1]]
        else:
            temp_list2=[{}]
        temp_dict1=temp_list1[0]
        temp_dict2=temp_list2[0]

        if(len(common_docs)==0):
            list1=list(temp_dict1.keys())
        else:
            list1=common_docs
        list2=list(temp_dict2.keys())
        ansList=andFunc(list1,list2)
        common_docs=ansList[0]
        
    
    documents=[]
    doc=0
    for index in common_docs:
        ans_temp_list=[]
        for i in range(0,len(token)-1):
            temp_list1=posDict[token[i]]
            temp_list2=posDict[token[i+1]]
            temp_dict1=temp_list1[0]
            temp_dict2=temp_list2[0]
            if (len(ans_temp_list)==0):
                posList1=temp_dict1[index]
            else:
                posList1=ans_temp_list
            posList2=temp_dict2[index]
        
            temp_list=[]
            for x in posList1:
                for y in posList2:
                    if x+1==y:
                        temp_list.append(y)
           
            if(len(temp_list)==0):
                ans_temp_list=temp_list
                break
            else:
                ans_temp_list=temp_list
        if(len(ans_temp_list)!=0):
            doc=doc+1
            documents.append(doctIdDict[index])
            
            

    queryCount=queryCount+1
    print("No of documents retrieved for query "+str(queryCount)+" using positional inverted index: ", doc)
    print("Names of document retrieved for query "+str(queryCount)+" using positional inverted index: ",documents)

In [26]:
n=int(input("Enter number of queries: "))
for i in range (n):
    userInput(i)
    userIP(i)

Enter Query: 
Tokens : ['wave boundary', 'boundary layer']
No of documents retrieved for query 1 using bigram inverted index:  3
Names of document retrieved for query 1 using bigram inverted index:  ['cranfield0002', 'cranfield0309', 'cranfield0569']
No of documents retrieved for query 1 using positional inverted index:  2
Names of document retrieved for query 1 using positional inverted index:  ['cranfield0002', 'cranfield0309']
Enter Query: 
Tokens : ['curved shock', 'shock wave']
No of documents retrieved for query 2 using bigram inverted index:  2
Names of document retrieved for query 2 using bigram inverted index:  ['cranfield0002', 'cranfield0401']
No of documents retrieved for query 2 using positional inverted index:  1
Names of document retrieved for query 2 using positional inverted index:  ['cranfield0002']
